# Building a Recommender Engine with PySpark ML

"User who liked ... also liked..." - nowadays, **recommender engines** are everywhere on the web. A recommender engine is basically any of a large variety of algorithms that recommends items to users while trying to maximize the likelyhood that the user will select them. This is also known as **collaborative filtering**, because such algorithms allow a user to use the input of many previous users to help them sift through the data.

In this example, we are going to build a simple recommender engine for movies. Given the ratings (1-5 stars) that a user has given to movies, the engine is going to predict the ratings that the user is likely to give to previously unseen movies.

## Preamble

In [ ]:
import findspark
findspark.init()
import pyspark

## Loading the Data

Our training data comes from the [MovieLens](https://grouplens.org/datasets/movielens/) dataset.

In [ ]:
data_dir = "../.assets/data/movielens/small"

In [ ]:
!ls {data_dir}

In [ ]:
!head {data_dir}/movies.csv

In [ ]:
!head {data_dir}/ratings.csv



After creating a `SparkSession`, we read the contents of the `movies.csv` and `ratings.csv` files into a DataFrame each.

In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("Movie Recommender") \
    .getOrCreate()


In [ ]:
movies = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema("movieId INT, title STRING, genres STRING") \
    .load(f"{data_dir}/movies.csv") 


In [ ]:
movies.show()

In [ ]:
ratings = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema("userId INT, movieId INT, rating FLOAT, timestamp INT") \
    .load(f"{data_dir}/ratings.csv") 


In [ ]:
ratings.show()

In [ ]:
ratings = ratings.drop("timestamp")

For simplicity of interpretation, we append each movie's title by joining the movies dataframe to the ratings dataframe:

In [ ]:
ratings = ratings.join(movies, on="movieId")

In [ ]:
ratings.show()

## Training a Recommendation Model

Building a rudimentary recommender engine is now as simple as fitting one of the algorithms from the `pyspark.ml.recommendation` module to a part of the ratings data defined as the training set.

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.recommendation import ALS

In [ ]:
# Build the recommendation model using ALS on the training data
# Note 
als = ALS(userCol="userId",
          itemCol="movieId",
          ratingCol="rating",
          coldStartStrategy="drop", # we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
         )
model = als.fit(training)

## Evaluating the Model

In [ ]:
model.transform(test).show()

### Error Metrics

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

Evaluate the model by computing the RMSE on the test data:


In [ ]:
predictions = model.transform(test)

In [ ]:
mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction").evaluate(predictions)

In [ ]:
rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction").evaluate(predictions)

In [ ]:
mae

In [ ]:
rmse

### Example Recommendations

As a sanity check, let's pick out a user and look at their ratings and the recommendations generated:

In [ ]:
user = 42

In [ ]:
training[training["userId"] == user].sort("rating", ascending=False).show(50, truncate=True)

In [ ]:
predictions[predictions["userId"] == user].sort("prediction", ascending=False).show(truncate=True)

## So how does it work actually?

In this course we do not go deep into the mathematics or algorithmics of machine learning, but since you asked: The ALS algorithm used above uses a mathematical technique called **matrix factorization**. [This blogpost](https://beckernick.github.io/matrix-factorization-recommender/) explains the approach, also using the movie ratings data set. As usual in machine learning, matrix factorization entails an optimization problem, and **alternating least squares** is a fast and parallelizable way of solving it, as [explained here](https://www.quora.com/What-is-the-Alternating-Least-Squares-method-in-recommendation-systems-And-why-does-this-algorithm-work-intuition-behind-this).

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2025 [Point 8 GmbH](https://point-8.de)_